# Game Setup for Studying Non-verbal Collaboration
Author: Saber Sheybani

Advisor: Eatai Roth

Indiana University

Summer 2018

# Current Status
- The objects are created: cursor, control points, targets.
- The CPs move 2px/stroke and the cursor's x position is a weighted average of the other two.
- Start a timer from the beginning of the game until the cursor reaches the target.
- When target1 (the left one) is reached, report the time elapsed and exit.
- Generate a file that logs the Reaching Time
- Reset the game.
- Define proper classes for the control points and the cursor
- config.yml and protocol.yml are created as such:
    - 1- The configuration file: The parameters that we iterate on in our course of studies: e.g. The position of targets. The speed that the cursor 
    - it will contain default values for some of the parameters that we might wanna override in protocol file multiple times.
    - Also the parameters related to the appearance, which will probably remain fixed.

    - 2-Protocol file: changes in a given instantiation of the experiment. The bulk of the 
     changing params.
- Our current quit function is the except block, which closes the log file (saves the 
    results properly.)
- The cursor movement is saved in mouse_trajectory.csv. The trajectory is also plotted.
- The game is 2D now, i.e. the targets are trajectories. They're read from a CSV file. Note that the target trajectories can be created simply by drawing them with the mouse and extracting them from mouse_trajectory.csv file.
- Timestamp is saved along with cursor trajectory.
- The cursor is controlled by joysticks


    
# Add the following features:


- A score for how closely the target was followed. Ideally we want to accumulate the distance between the target and the cursor. However, depending on the shape of the target, this can get very complicated. It's probably much easier to implement it after deciding about the shape of the target.

- Implement the game with the target trajectory being a circle. The mapping for the two players should be as such:
    P1: x=u
    y= 0.5v

    P2: x=0.5 u
    y= v
- Limit the visible part of the trajectory to a block around the cursor. (In order to limit planning of the movements) Decide about it!! (See the subsection below)

- Change scaling. Keep track of fractional position, but display rounded pixel positions of cursor.

- Size of screen. Colors. Line thickness.

- Design the paths to follow. (Getting increasingly harder)
    - level 1-circle
    - x=sin(w1 t), y = cos(w2 t)
    - x= sin () + cos(), y =sin () + cos()

- The past parts of the trajectory should disappear.

by friday -------------
- One csv file having 7 columns as such: time, joystick1 pos(x,y), joystick2 pos (x,y), cursor pos, 
- Also a header which explains the parameters of the experiment.


- Design training and testing sequence. e.g. participant plays alone to learn their own input-output model, pairs of participants share control

- Hypotheses for collaboration
    - one-person does it all
    - speciallization of roles
    - everybody does the same thing
    
- Feedback/reward through score. sometime to enforce speciallization. do the players need it?



# Existing Issues
- An error after finishing the game
- With the current mappings, the cursor moves too fast to allow fine control of the trajectory.
    - Use an intermediate variable, to accumulate the movement.

## Unimportant issues
- Turns out it's impossible to treat the two mice separately in Pygame. It can be done with joysticks though!
- Note the size of the final screen. Currently the game is only set to work on 768x1024.


## Code Improvements:
    - The time.tick does not reset in the beginning of the game. but that's not a big problem.
    
## Notes
- The pygame.time.delay doesn't work as expected for avoiding multiple button presses.
 Used event.keydown instead.


### About Limiting the visible part of the trajectory
Problem:
PyGame gives us a function, draw.aalines, for drawing a trajectory with a list of points. 
We have a list of points forming the target trajectory (in an arbitrary resolution, currently 768x1024). 
At each iteration of the game, we want to:
    1- Select a subset of the points,
    2- Draw them on the screen properly in another scale. This requires applying 
    a 1-1 mapping to the points.    
        This includes rotating the trajectory the same way as in car racing games.
    3- Calulate the performance. If we later want to have a good representation (e.g. plot) of the cursor trajectory, applying the reverse of the original mapping might be necessary.


#### The mapping
- Calculate the mean of the points. Align that with the middle of the screen.
- Scale the points related to the axis by a fixed factor, e.g. 3 or 4.

The issue with the above approach is that taking the cursor trajectory back to the original coordinates will be hard/problematic. The same problem may also happen for the other kinds of processing that we do in the future.

A whole different approach would be not doing any of these. Instead just block the other areas of the trajectory. In this case, in order to avoid giving cues to the user,
the size of the whole trajectory should be relatively small compared to the size of the screen.


In [27]:
import pygame as pg
import time #@@@@@@@ Only cause the pygame delay doesn't work properly, 
    #keeping the program running in the background!
#from pygame.locals import *
from pygame import display as pgdisp
import numpy as np
import yaml #For parsing config files
import csv
import matplotlib.pyplot as plt
import numpy as np
#import pygame.display
#import pygame.image

In [28]:
#Joystick Configuration

# Note that the joystick devices should be connected when starting the jupyter notebook. 
# if disconnected in the middle, the pg.joystick will not realize and its methods return wrong outputs.
pg.joystick.init()
joysticks = [pg.joystick.Joystick(x) for x in range(pg.joystick.get_count())]
joysticks[0].init()
joysticks[1].init()
if joysticks[0].get_init() and joysticks[1].get_init():
    pass
else:
    raise IOError
    
#yaml.load('zd_macros.yml')
#Defines the following parameters
X=0
Y=1

In [29]:
#Experiment Parameters

#yaml.load('config.yml')
#Defines the following parameters
# Color codes
WHITE=[255, 255, 255]
WOOD=[139, 90, 43]
RED=[255, 0, 0]
GREEN=[0, 255, 0]

#Specify the graphical attributes of the window and the objects
WIDTH = 1024
HEIGHT = 768
background_color = WOOD


cp_icon_path = 'cp_icon.png'
cursor_icon_path = 'plus_cursor.png'

FRAME_RATE = 30

WAIT_TIME_AFTER_FINISH = 2000

#yaml.load('protocol.yml')
# Defines the following parameters
cp1_init_pos = [WIDTH//2,1*HEIGHT//6]
cp2_init_pos = [WIDTH//2,5*HEIGHT//6]
cursor_init_pos = [WIDTH//2, HEIGHT//2]

targetfocusR = 40
    
# Mappings.
MAP1=[1, 0.5] #Mapping from player 1 input to cursor.
MAP2=[0.5, 1] #Mapping from player 2 input to cursor.

In [30]:
MovingObjects = pg.sprite.Group()

class ControlPoint(pg.sprite.Sprite):

    # Constructor. Pass in its x and y position
    def __init__(self, pos):
       # Call the parent class (Sprite) constructor and also the MovingObjects group
       pg.sprite.Sprite.__init__(self, MovingObjects)
    
       self.rect = pg.Rect(0, 0, 10, 10)
       self.rect.center=[pos[0],pos[1]] 
       self.image = pg.image.load(cp_icon_path)

        
class Cursor(pg.sprite.Sprite):

    # Constructor. Pass in its x and y position
    def __init__(self, pos):
       # Call the parent class (Sprite) constructor and also the MovingObjects group
       pg.sprite.Sprite.__init__(self, MovingObjects)

       self.rect = pg.Rect(0, 0, 30, 32)
       self.rect.center=[pos[0],pos[1]] 
       self.image = pg.image.load(cursor_icon_path)

In [31]:
screen = pgdisp.set_mode((WIDTH, HEIGHT))
background = pg.Surface(screen.get_size())
background.fill(background_color)
pgdisp.update()

In [32]:
# Target

target_pts=[]
with open('target_trajectory.csv') as f_traj:
    readCSV = csv.reader(f_traj, delimiter=',')
    for row in readCSV:
        target_pts.append(list(map(int,row)))

In [33]:
# Initialize target and Current Cursor Position
def init_target():
    # Create target_focus vector
    target_kernel = [0,0]
    for i in range(len(target_pts)):
        if np.linalg.norm(np.asarray(target_pts[i])-np.asarray(target_pts[0])) <targetfocusR:
    #for point in target_pts:
        #if np.linalg.norm(np.asarray(point)-np.asarray(target_pts[0])) <targetfocusR:
            target_kernel[1] = i
        else:
            break
    return target_kernel

In [34]:
def update_target(cursor_pos, t_kernel):
    # Find the 10 closet points to the cursor_pos

    focusC = cursor_pos
    
    while np.linalg.norm(np.asarray(target_pts[t_kernel[1]])-np.asarray(cursor_pos)) <targetfocusR and \
          t_kernel[1]<len(target_pts):
        t_kernel[1] +=1
        
        
    #for i in range(len(target_pts)):
    #while np.linalg.norm(np.asarray(target_pts[t_kernel[0]])-np.asarray(cursor_pos)) >targetfocusR and \
    #      t_kernel[0] < t_kernel[1]-1:
    #    t_kernel[0] +=1
    #else:
    #    break
        

    return t_kernel
    # put them in target_curr
    #target_traj = pg.draw.aalines(background, GREEN, True, target_curr, 1)

In [35]:
def run_game(cursor):
    #Read parameters from the protocol file
    # Create target focus vector
    target_kernel = init_target()
    target_traj = pg.draw.aalines(background, GREEN, False, 
                                  target_pts[target_kernel[0]:target_kernel[1]], 1)
    
    # create the obligatory event loop
    clock = pg.time.Clock()
    timestamp = [pg.time.get_ticks()]
    cursor_traj=[tuple(cursor.rect.center)]
    score = 0
    Running=True
    
    #print("starting while running!")
    
    while Running == True:
        # limit runtime speed to 30 frames/second
        clock.tick(FRAME_RATE)
        pg.event.pump()  #allow pygame to handle internal actions.maybe unnecessary!
        
        # Check for events on keyboard
        events = pg.event.get()
        for event in events:
            if event.type == pg.KEYDOWN:
                if event.key == pg.K_RETURN:  #Exit with saving
                    pg.time.delay(WAIT_TIME_AFTER_FINISH)
                    Running = False
                    MovingObjects.clear(screen, background)
                    break
                    
                elif event.key == pg.K_ESCAPE:  #Exit without saving
                    Running = False
                    raise SystemExit
            
            elif event.type == pg.QUIT:
                pg.quit()
                raise SystemExit
        """            
        if keyinput[pg.K_a]:
            #press a key to quit with saving timestamp and cursor trajectory
            print('a pressed!')
            #pg.time.delay(WAIT_TIME_AFTER_FINISH)
            time.sleep(2)
            Running = False
            MovingObjects.clear(screen, background)
            break
        # press escape key to quit game
        if keyinput[pg.K_ESCAPE]:
            print("Esc pressed!")
            Running = False
            raise SystemExit
            #Running=False
        # optional exit on window corner x click
        for event in pg.event.get():

      """
        # Translate the movement from joysticks to the cursor
        cursor.rect.centerx += 4*(joysticks[0].get_axis(X)*MAP1[0] + \
                               joysticks[1].get_axis(X)*MAP2[0])
        cursor.rect.centery += 4*(joysticks[0].get_axis(Y)*MAP1[1] + \
                               joysticks[1].get_axis(Y)*MAP2[1])
        #Update background and cursor position
        MovingObjects.clear(screen, background)
        MovingObjects.draw(screen)
        #Update target trajectory
        target_kernel = update_target(cursor.rect.center, target_kernel)
        pg.draw.aalines(screen, GREEN, False,
                        target_pts[target_kernel[0]:target_kernel[1]], 1)

        # Save changes
        timestamp.append(pg.time.get_ticks())
        cursor_traj.append(tuple([cursor.rect.centerx, cursor.rect.centery]))


        # update display
        pg.display.flip()
        if Running==False:
            break
        #raise RuntimeError
    return timestamp, cursor_traj, score

In [36]:
def game_init():

    # Create the cursor
    cursor = Cursor(cursor_init_pos)
    cursor.rect.center = target_pts[0]

    return cursor

In [37]:


#Move the Control  points

file_log = open('game_log.dat','w')

try:
    while 1:
        #print("started while 1")
        cursor = game_init()
        
        #Display the window
        screen.blit(background, (0, 0))
        #Display the moving objects
        MovingObjects.draw(screen)
        pg.display.set_caption("Move with the joysticks (press Escape to exit)")
        #target_traj = pg.draw.aalines(background, GREEN, True, target_focus, 1)
        pgdisp.flip()
        
        timestamp, cursor_traj, score = run_game(cursor)
        #print("timestamps=", timestamp)
        #print("cursor_traj=", cursor_traj)
        file_log.write(str(timestamp[-1])+ '\n')

        with open('timestamp.csv','w') as f_time:
            wr = csv.writer(f_time)
            for point in timestamp:
                wr.writerow([point])

        with open('cursor_trajectory.csv','w') as f_traj:
            wr = csv.writer(f_traj)
            for point in cursor_traj:
                wr.writerow(point)
        MovingObjects.empty()
except SystemExit:
#except RuntimeError:
    file_log.close()
    pgdisp.quit()   
    pg.quit()
    print("Quitted the game!")


Quitted the game!


In [38]:
traj=[]
with open('cursor_trajectory.csv') as f_traj:
    readCSV = csv.reader(f_traj, delimiter=',')
    for row in readCSV:
        traj.append(list(map(int,row)))
        
traj=np.asarray(traj)

In [39]:
plt.figure()
plt.scatter(traj[:,0],traj[:,1])
plt.savefig('test1.png')